In [24]:
# Importing imp. libs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv('/content/train.csv')
df.shape

(404290, 6)

In [26]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [27]:
# As, this dataset is very big, I will take randomly 30k rows
new_df = df.sample(30000)

In [28]:
new_df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [29]:
new_df.duplicated().sum()

0

In [30]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
31363,How is life in Dubai?,How's life in Dubai?
337611,How would I go about starting Python?,Where should I start at to learn about how to ...
253563,What is the intuition behind Perceptron weight...,How do I implement SGD in Java after I have co...
335717,How do I make India as corruption free?,How can India be corruption free?
309245,Is the US election rigged?,Is the US general election rigged as Donald Tr...


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=30000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(), 2)
# isse mujhe har ques. ka BOW milgaya and split bhi kar diya

In [32]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)

temp_df.shape

(30000, 52376)

In [33]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,26178,26179,26180,26181,26182,26183,26184,26185,26186,26187
31363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
309245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
390526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138951,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [35]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26179,26180,26181,26182,26183,26184,26185,26186,26187,is_duplicate
31363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
337611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
253563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
309245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:, 0:-1].values, temp_df.iloc[:, -1].values, test_size=0.2, random_state=1)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7418333333333333

In [38]:
!pip install xgboost

In [39]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7228333333333333